In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# Add the locations of the data folders here, and the crop file locations
seasonal_data_folder = '../data/seasonal_era_agriclimatic/'
ten_day_data_folder = '../data/10-day-data/'
crop_data_2005_file = '../data/spam2005v3r2_global_yield/spam2005V3r2_global_Y_TA.csv'
crop_data_2010_file = '../data/spam2010v1r1_global_yield/spam2010V1r1_global_Y_TA.csv'

# Leave this, it just stores the names of the ten day feature files
ten_day_feature_files = {
    'BEDD': 'BEDD_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'FD': 'FD_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'R20mm': 'R20mm_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'R10mm': 'R10mm_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'ID': 'ID_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TG': 'TG_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TN': 'TN_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'DTR': 'DTR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'RR1': 'RR1_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'RR': 'RR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'SDII': 'SDII_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'SU': 'SU_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TG': 'TG_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TNn': 'TNn_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TR': 'TR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TX': 'TX_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TXn': 'TXn_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TXx': 'TXx_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc'
}

# I noticed the CSU file is actually the same as the CSDI file, so I havent included it
seasonal_feature_files = {
    'CDD': 'CDD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CFD': 'CFD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CWD': 'CWD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'WW': 'WW_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'WSDI': 'WSDI_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CSDI': 'CSDI_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc'
}

/home/edward_brown/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Create crop data file
crops = ['maize']
crop_data_2005 = pd.read_csv(crop_data_2005_file,  encoding = "ISO-8859-1")
crop_data_2010 = pd.read_csv(crop_data_2010_file,  encoding = "ISO-8859-1")

merged = crop_data_2010.merge(crop_data_2005, how='inner', on='alloc_key', suffixes=['_2010', '_2005'])
kept_columns = ['alloc_key', 'x', 'y', 'iso3_2010']
for crop in crops:
    kept_columns += [f'{crop[0:4]}_a_2010', f'{crop[0:4]}_a_2005']
merged = merged[kept_columns]
for crop in crops:
    merged = merged.rename(columns={
    f'{crop[0:4]}_a_2010': f'{crop}_a_2010',
    f'{crop[0:4]}_a_2005': f'{crop}_a_2005',
})
    
merged = merged.rename(columns={
    'iso3_2010': 'iso3',
    'x': 'lon',
    'y': 'lat'
})

del crop_data_2005
del crop_data_2010
data = merged.dropna()
del merged

# Only include non zero yields
data = data[data['maize_a_2005'] > 0]
data = data[data['maize_a_2010'] > 0]
data

,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005
0,4383640,123.291667,53.541667,CHN,3918.1,2819.2
2,4393629,122.375000,53.458333,CHN,3119.3,2028.5
7,4403648,123.958333,53.375000,CHN,3230.1,2337.9
8,4403649,124.041667,53.375000,CHN,3119.3,2010.2
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2
...,...,...,...,...,...,...
665862,7670982,-98.208333,26.125000,USA,5939.7,6106.9
665863,7670983,-98.125000,26.125000,USA,6050.9,6107.8
665864,7670984,-98.041667,26.125000,USA,6111.0,6131.7
665865,7670985,-97.958333,26.125000,USA,6152.1,6181.2


In [3]:
# This makes columns which compute the nearest the x y coordinates in the climate data.
# This massively speeds up the computation of the climatic indicators.
data['nearest_lat'] = 0
data['bool'] = 0
data['bool'] = ((data['lat'] % 1) <= 0.5).astype(float)
data['nearest_lat'] =  data['bool'] * (data['lat'].astype(int) + 0.25) + (1 - data['bool']) * (data['lat'].astype(int) + 0.75)

data['nearest_lon'] = 0
data['bool'] = 0
data['bool'] = ((data['lon'] % 1) <= 0.5).astype(float)
data['nearest_lon'] =  data['bool'] * (data['lon'].astype(int) + 0.25) + (1 - data['bool']) * (data['lon'].astype(int) + 0.75)
data = data.drop(columns='bool')
data

,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,nearest_lat,nearest_lon
0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,53.75,123.25
2,4393629,122.375000,53.458333,CHN,3119.3,2028.5,53.25,122.25
7,4403648,123.958333,53.375000,CHN,3230.1,2337.9,53.25,123.75
8,4403649,124.041667,53.375000,CHN,3119.3,2010.2,53.25,124.25
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,53.25,123.25
...,...,...,...,...,...,...,...,...
665862,7670982,-98.208333,26.125000,USA,5939.7,6106.9,26.25,-97.25
665863,7670983,-98.125000,26.125000,USA,6050.9,6107.8,26.25,-97.25
665864,7670984,-98.041667,26.125000,USA,6111.0,6131.7,26.25,-97.25
665865,7670985,-97.958333,26.125000,USA,6152.1,6181.2,26.25,-96.75


In [7]:
# Compute the seasonal features
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
years = ['2010', '2005']
seasonal_features = [
    'CDD',
    'CFD',
    'CWD',
    'WW',
    'WSDI',
    'CSDI'
]
quarter_time_mapping = {
    'Q1': '01-16',
    'Q2': '04-16',
    'Q3': '07-16',
    'Q4': '10-16'
}

lats_ = xr.DataArray(list(data['lat'].values), dims='z')
lons_ = xr.DataArray(list(data['lon'].values), dims='z')

for feature in tqdm(seasonal_features):
    for year in years:
        with xr.open_dataset(seasonal_data_folder + seasonal_feature_files[feature]) as ds:
            feature_data = ds.load()
        for quarter in quarters:
            feature_name = f'{feature}-{quarter}-{year}'
            time = f'{year}-{quarter_time_mapping[quarter]}'
            time_data = feature_data.interp(time=time, lat=lats_, lon=lons_, method='linear')
            data[feature_name] = getattr(time_data, feature)
            del time_data
    del feature_data
    
data

100%|██████████| 6/6 [00:18<00:00,  3.16s/it]


,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,nearest_lat,nearest_lon,CDD-Q1-2010,CDD-Q2-2010,...,WSDI-Q3-2005,WSDI-Q4-2005,CSDI-Q1-2010,CSDI-Q2-2010,CSDI-Q3-2010,CSDI-Q4-2010,CSDI-Q1-2005,CSDI-Q2-2005,CSDI-Q3-2005,CSDI-Q4-2005
0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,53.75,123.25,27.0000,29.194444,...,0.00,0.0000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,4393629,122.375000,53.458333,CHN,3119.3,2028.5,53.25,122.25,27.0000,24.604167,...,0.00,0.0000,0.0,0.0,0.0,0.0,3.375,0.0,0.0,0.0
7,4403648,123.958333,53.375000,CHN,3230.1,2337.9,53.25,123.75,24.8125,26.000000,...,0.00,0.0000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
8,4403649,124.041667,53.375000,CHN,3119.3,2010.2,53.25,124.25,23.9375,26.000000,...,0.00,0.0000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,53.25,123.25,27.0000,26.326389,...,0.00,0.0000,0.0,0.0,0.0,0.0,2.500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665862,7670982,-98.208333,26.125000,USA,5939.7,6106.9,26.25,-97.25,16.0000,23.458333,...,9.25,10.6875,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
665863,7670983,-98.125000,26.125000,USA,6050.9,6107.8,26.25,-97.25,16.0000,23.375000,...,9.25,10.5625,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
665864,7670984,-98.041667,26.125000,USA,6111.0,6131.7,26.25,-97.25,16.0000,23.291667,...,9.25,10.4375,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
665865,7670985,-97.958333,26.125000,USA,6152.1,6181.2,26.25,-96.75,16.0000,23.208333,...,9.25,10.3125,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0


In [9]:
# Compute the 10 day features

# Choose the features you want at the top of the notebook based on the ten_day_feature_files
# dict.
days = ['05', '15', '25']
years = ['2010', '2005']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
ten_day_features = [
    'BEDD',
    'FD',
    'R20mm',
    'RR1',
    'DTR'
]

lats_ = xr.DataArray(list(data['lat'].values), dims='z')
lons_ = xr.DataArray(list(data['lon'].values), dims='z')

for feature in tqdm(ten_day_features):
    with xr.open_dataset(ten_day_data_folder + ten_day_feature_files[feature]) as ds:
        feature_data = ds.load()
    for day in days:
        for month in months:
            for year in years:
                time = f'{year}-{month}-{day}'
                time_data = feature_data.interp(time=time, lat=lats_, lon=lons_, method='linear')
                data[f'{feature}-{month}-{day}-{year}'] = getattr(time_data, feature)
    del feature_data
    del time_data
data


100%|██████████| 5/5 [08:12<00:00, 98.50s/it]


,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,nearest_lat,nearest_lon,CDD-Q1-2010,CDD-Q2-2010,...,DTR-08-25-2010,DTR-08-25-2005,DTR-09-25-2010,DTR-09-25-2005,DTR-10-25-2010,DTR-10-25-2005,DTR-11-25-2010,DTR-11-25-2005,DTR-12-25-2010,DTR-12-25-2005
0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,53.75,123.25,27.0000,29.194444,...,15.398716,14.092479,11.727703,15.016566,13.787956,13.988310,9.926498,18.100036,11.371226,12.329345
2,4393629,122.375000,53.458333,CHN,3119.3,2028.5,53.25,122.25,27.0000,24.604167,...,15.253838,14.457341,11.912833,15.568386,13.628348,14.757354,11.160431,18.749596,11.007959,12.080565
7,4403648,123.958333,53.375000,CHN,3230.1,2337.9,53.25,123.75,24.8125,26.000000,...,15.753938,14.120732,11.412658,14.818996,13.940518,13.382273,9.111539,17.323588,11.555232,12.099928
8,4403649,124.041667,53.375000,CHN,3119.3,2010.2,53.25,124.25,23.9375,26.000000,...,15.751151,14.092668,11.393748,14.805402,13.889010,13.276832,8.999466,17.190924,11.520067,12.048447
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,53.25,123.25,27.0000,26.326389,...,15.442677,14.253544,11.628774,15.248663,14.388440,14.395436,10.273176,18.549792,11.431268,11.976130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665862,7670982,-98.208333,26.125000,USA,5939.7,6106.9,26.25,-97.25,16.0000,23.458333,...,13.231907,13.562503,11.222518,18.798829,13.552721,14.522762,14.321142,17.132996,13.517107,14.743057
665863,7670983,-98.125000,26.125000,USA,6050.9,6107.8,26.25,-97.25,16.0000,23.375000,...,13.051782,13.341714,11.200690,18.607473,13.399567,14.560697,14.159338,16.967525,13.315258,14.500550
665864,7670984,-98.041667,26.125000,USA,6111.0,6131.7,26.25,-97.25,16.0000,23.291667,...,12.871656,13.120926,11.178863,18.416116,13.246412,14.598632,13.997534,16.802054,13.113409,14.258044
665865,7670985,-97.958333,26.125000,USA,6152.1,6181.2,26.25,-96.75,16.0000,23.208333,...,12.691531,12.900137,11.157036,18.224760,13.093258,14.636568,13.835730,16.636583,12.911560,14.015538


In [10]:
# Save data to a csv
file_path = f'head_of_soils_recommendations.csv'
data.to_csv(file_path)

In [ ]:
# Inspect a CSV file
file_path = f'head_of_soils_recommendations.csv'
data = pd.read_csv(file_path)
data.head()

In [9]:
# Create a custom train and test set adding in the features you want. It reads an already computed 
# dataset but you can then remove features to see the effect on score.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random

# Which data set are we reading in?
read_file_path = f'head_of_soils_recommendations.csv'

# Set the random seed for the whole process of creating the training and test set.
random_seed = 42
random.seed(a=random_seed)
np.random.seed(random_seed)

# Test set fraction. What fraction of the data set to make a test set
test_size = 0.2

# Only read a sample fraction of the data, speeds things up, but you'll get worse performance.
# Use 1.0 as the sample fraction to get the full data set
sample_fraction = 1.0
data = pd.read_csv(read_file_path, skiprows=lambda i: i>0 and random.random() > sample_fraction)

# Remove datapoints where yield changes by more than the fraction between 2005 and 2010.
# This is unlikely to be due to changes in climate.
limit_change_in_yield = False
limit = 0.5

# Which type of features to include (if True must already exist in read dataset)
include_coordinates = True
include_2005_maize = True
include_seasonal = True
include_ten_day = True
include_2005_climate = True
include_soil_type = False
include_growing_zones = False

# Drop nans?
drop_nans = True

# Choose a subset of your features! (must already exist in original file)
ten_day_features = [
    'BEDD',
    'R20mm',
    'DTR',
    'RR1',
    'FD'
]

seasonal_features = [
    'CDD',
    'CFD',
    'CWD',
    'WW',
    'WSDI',
    'CSDI'
]

if limit_change_in_yield:
    data = data[abs(data['maize_a_2010'] - data['maize_a_2005']) / data['maize_a_2005'] < limit]

features = []

if include_growing_zones:
    growing_zones = [
        'Inland water bodies',
        'Subtropics - summer rainfall',
        'Subtropics - winter rainfall',
        'Temperature - continental',
        'Temperature - oceanic',
        'Temperature - subcontinental',
        'Tropics'
    ]
    
if include_soil_type:
    features = features + ['soil_type']

if include_2005_climate:
    years = ['2010', '2005']
else:
    years = ['2010']

if include_seasonal:
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    for feature in seasonal_features:
        for quarter in quarters:
            for year in years:
                features.append(f'{feature}-{quarter}-{year}')

if include_ten_day:
    days = ['05', '15', '25']
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    for feature in ten_day_features:
        for day in days:
            for month in months:
                for year in years:
                    features.append(f'{feature}-{month}-{day}-{year}')

if include_coordinates:
    features = features + ['lon', 'lat']
    
if include_2005_maize:
    features = features + ['maize_a_2005']

target = 'maize_a_2010'

if drop_nans:
    data = data.dropna()
    
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_seed)

del X
del y
del data

/home/edward_brown/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
# Try a simple random forest regressor
regressor = RandomForestRegressor(n_estimators=100, n_jobs=10, random_state=random_seed, verbose=3)
regressor.fit(X_train, y_train)
score = regressor.score(X_test, y_test)
print(score)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100building tree 7 of 100

building tree 8 of 100
building tree 9 of 100building tree 10 of 100

building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100


[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:  3.8min


building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63

[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed: 20.3min finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    0.2s


0.9754846093311359


[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    1.1s finished


In [ ]:
# Predict on Hadgem data